In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix

In [4]:
###Loading data

from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets("data/MNIST/",one_hot=True) ##ImP ** one_hot is true

Extracting data/MNIST/train-images-idx3-ubyte.gz
Extracting data/MNIST/train-labels-idx1-ubyte.gz
Extracting data/MNIST/t10k-images-idx3-ubyte.gz
Extracting data/MNIST/t10k-labels-idx1-ubyte.gz


In [9]:
print("Size of:")
print("Training Set:\t\t{}".format(len(data.train.labels)))
print("Test Set:\t\t{}".format(len(data.test.labels)))
print("Validation Set:\t\t{}".format(len(data.validation.labels)))

Size of:
Training Set:		55000
Test Set:		10000
Validation Set:		5000


In [10]:
data.test.labels[0:5,:] ## labels in Hot encoding

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.]])

In [11]:
data.test.cls = np.array([label.argmax() for label in data.test.labels])

In [13]:
data.test.cls[0:5] ## This now holds cls which represent label class

array([7, 2, 1, 0, 4], dtype=int64)

In [ ]:
img_size = 28   ##  as there are 28 pixels in each dimension

img_size_flat = img_size * img_size  ## Now entire image is extended to this number of features

img_shape = (img_size,img_size)

num_classes = 10

In [36]:
x = tf.placeholder(tf.float32,[None,img_size_flat]) ## None represents number of training examples

y_true = tf.placeholder(tf.float32,[None,num_classes]) ## This is in HOT ENCODING form

y_true_cls = tf.placeholder(tf.int64,[None]) ## 

weigths = tf.Variable(tf.zeros([img_size_flat,num_classes])) ## IMP ** Initialized to zeros

biases = tf.Variable(tf.zeros([num_classes]))

In [37]:
logits = tf.add(tf.matmul(x,weigths),biases) 

y_pred = tf.nn.softmax(logits) ## In the last layer softmax function is applied

y_pred_cls  = tf.argmax(y_pred,axis=1)

In [86]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=y_true) ##IMP** HOt ENcoding are passed 

cost  = tf.reduce_mean(cross_entropy) ## Cross_entropy is a vector and here we are calculating average

optimizer = tf.train.AdagradOptimizer(learning_rate=0.5).minimize(cost)

In [87]:
correct_prediction = tf.equal(y_pred_cls, y_true_cls)

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [88]:
session = tf.Session()

session.run(tf.global_variables_initializer())

In [89]:
batch_size = 1000

def optimize(num_iterations):
    for i in range(num_iterations):
        
        x_batch,y_true_batch = data.train.next_batch(batch_size)
        feed_dict = {x:x_batch,y_true:y_true_batch}
        
        session.run(optimizer,feed_dict = feed_dict)
        

In [90]:
feed_dict_test = {x: data.test.images,
                  y_true: data.test.labels,
                  y_true_cls: data.test.cls}

def print_accuracy():
    acc = session.run(accuracy, feed_dict=feed_dict_test)
    
    print("Accuracy on test-set: {0:.1%}".format(acc))
    
def print_confusion_matrix():
    cls_true = data.test.cls
    
    cls_pred = session.run(y_pred_cls, feed_dict=feed_dict_test)

    cm = confusion_matrix(y_true=cls_true,
                          y_pred=cls_pred)

    print(cm)
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)

    plt.tight_layout()
    plt.colorbar()
    tick_marks = np.arange(num_classes)
    plt.xticks(tick_marks, range(num_classes))
    plt.yticks(tick_marks, range(num_classes))
    plt.xlabel('Predicted')
    plt.ylabel('True')
    
def plot_example_errors():
    correct, cls_pred = session.run([correct_prediction, y_pred_cls],
                                    feed_dict=feed_dict_test)

    incorrect = (correct == False)
    images = data.test.images[incorrect]
    
    cls_pred = cls_pred[incorrect]

    cls_true = data.test.cls[incorrect]
    
    plot_images(images=images[0:9],
                cls_true=cls_true[0:9],
                cls_pred=cls_pred[0:9])
    
def plot_weights():
    w = session.run(weights)
    
    w_min = np.min(w)
    w_max = np.max(w)

    fig, axes = plt.subplots(3, 4)
    fig.subplots_adjust(hspace=0.3, wspace=0.3)
    
    for i, ax in enumerate(axes.flat):
        # Only use the weights for the first 10 sub-plots.
        if i<10:
            image = w[:, i].reshape(img_shape)

            ax.set_xlabel("Weights: {0}".format(i))

            ax.imshow(image, vmin=w_min, vmax=w_max, cmap='seismic')

        ax.set_xticks([])
        ax.set_yticks([])


In [92]:
optimize(num_iterations=10000)
print_accuracy()

Accuracy on test-set: 92.6%
